In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import itertools
import time

In [2]:
def before_after(a, total_width):
    before = (total_width-a)//2
    after = total_width-before-a
    return (before, after)


def get_pad_width(x,y,z,total_width=400):
    return (before_after(x,total_width),
            before_after(y,total_width),
            before_after(z,total_width))


def get_padded_data(data, width=400):
    x,y,z = list(data.shape)
    return np.pad(data,get_pad_width(x,y,z),'constant',constant_values=0)


def get_block_means(data, width=400, num_blocks=4):
    block_means=[]
    ix = []
    for i in range(num_blocks):
        ix.append([width // num_blocks * i, width // num_blocks * (i+1)])
    count = 0
    for i, j, k in itertools.product(ix,ix,ix):
        block_means.append(np.mean(data[i[0]:i[-1],j[0]:j[-1],k[0]:k[-1]]))
    return block_means


def get_centered_block_means(data, width=400, num_blocks=10):
    ix = []
    for i in range(num_blocks):
        half_width = width//2
        half_region = width//num_blocks//2*(i+1)
        ix.append([half_width-half_region, half_width+half_region])
    centered_means = []
    for i in ix:
        centered_means.append(np.mean(padded[i[0]:i[-1],i[0]:i[-1],i[0]:i[-1]]))
    return centered_means

In [3]:
df = pd.read_csv('../data/ATR_GT_Training.csv', header=None, names=['file','label'])
df.file = df.file.map(lambda x: x.replace("'",''))
df['img'] = df.file.map(lambda x: nib.load('../data/' + str(x) + '.nii.gz'))

In [4]:
blocks = 4
block_labels = ['block_' + str(x) + '_mean' for x in range(1,blocks**3+1)]
centered_blocks = 10
centered_block_labels = ['centered_block_' + str(x) + '_mean' for x in range(1,centered_blocks+1)]
new_features = block_labels + centered_block_labels
df = df.reindex(columns = df.columns.tolist() + new_features)

In [7]:
start = time.time()
start100 = start
features_regional = []
for i, img in enumerate(df.img):
    if i % 25 == 0 and i != 0:
        now = time.time()
        print('i = {:<10d}time for last 25 = {:<10.3g}total time = {:<10.3g}'.format(i,now-start100,now-start))
        start100 = time.time()
    padded = get_padded_data(img.get_fdata())
    region_means = get_block_means(padded)
    centered_means = get_centered_block_means(padded)
    features = region_means + centered_means
    features_regional.append(features)

print('Done')
np.save('features_regional.npy', np.array(features_regional))

i = 25        time for last 25 = 22.7      total time = 22.7      
i = 50        time for last 25 = 22        total time = 44.7      
i = 75        time for last 25 = 21.6      total time = 66.3      
i = 100       time for last 25 = 22.1      total time = 88.4      
i = 125       time for last 25 = 21.5      total time = 110       
i = 150       time for last 25 = 22.6      total time = 132       
i = 175       time for last 25 = 22.1      total time = 155       
i = 200       time for last 25 = 21.2      total time = 176       
i = 225       time for last 25 = 21.3      total time = 197       
i = 250       time for last 25 = 21.8      total time = 219       
i = 275       time for last 25 = 22.4      total time = 241       
i = 300       time for last 25 = 22.1      total time = 263       
i = 325       time for last 25 = 22.7      total time = 286       
i = 350       time for last 25 = 21.3      total time = 307       
i = 375       time for last 25 = 20.9      total time = 328   